# Train an ML Model using Apache Spark in EMR and deploy in SageMaker
In this notebook, we will see how you can train your Machine Learning (ML) model using Apache Spark and then take the trained model artifacts to create an endpoint in SageMaker for online inference. Apache Spark is one of the most popular big-data analytics platforms & it also comes with an ML library with a wide variety of feature transformers and algorithms that one can use to build an ML model. 

Apache Spark is designed for offline batch processing workload and is not best suited for low latency online prediction. In order to mitigate that, we will use [MLeap](https://github.com/combust/mleap) library. MLeap provides an easy-to-use Spark ML Pipeline serialization format & execution engine for low latency prediction use-cases. Once the ML model is trained using Apache Spark in EMR, we will serialize it with `MLeap` and upload to S3 as part of the Spark job so that it can be used in SageMaker in inference.

After the model training is completed, we will use SageMaker **Inference** to perform predictions against this model. The underlying Docker image that we will use in inference is provided by [sagemaker-sparkml-serving](https://github.com/aws/sagemaker-sparkml-serving-container). It is a Spring based HTTP web server written following SageMaker container specifications and its operations are powered by `MLeap` execution engine. 

In the first segment of the notebook, we will work with `Sparkmagic (PySpark)` kernel while performing operations on the EMR cluster and in the second segment, we need to switch to `conda_python2` kernel to invoke SageMaker APIs using `sagemaker-python-sdk`.

## Setup an EMR cluster and connect a SageMaker notebook to the cluster
In order to perform the steps mentioned in this notebook, you will need to have an EMR cluster running and make sure that the notebook can connect to the master node of the cluster. 

**The guide in the next paragraph does not include that you have to add the ec2 key par to the cluster in order to be able to connect via ssh. See the guide https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html#having-ec2-create-your-key-pair how you can create one.**

**At this point, `sagemaker-sparkml-serving` only supports models trained with Spark version 2.2 for performing inference. Hence, please create an EMR cluster with Spark 2.2.0 or Spark 2.2.1 if you want to use your Spark ML model for online inference or batch transform.**

Please follow the guide here on how to setup an EMR cluster and connect it to a notebook.
https://aws.amazon.com/blogs/machine-learning/build-amazon-sagemaker-notebooks-backed-by-spark-in-amazon-emr/ .


This notebook is written in Python2, but you should be able to use Python3 with minimal changes in the instruction here. Python2 or 3 has no impact on the model serialization or inference. 

## Install additional Python dependencies and JARs in the EMR cluster
In order to serialize a Spark model with `MLeap` and upload to S3, we will need some additional Python dependencies and JAR present in the EMR cluster. Also, you need to setup your cluster with proper aws configurations.

### Conect to the cluster

1. download emrkey

2. add permission:

`chmod 400 emr.pem`

3. check key (this step is not mandatory): 

`ssh-keygen -y -f emr.pem`

4. connect to the cluster using public DNS:

`ssh hadoop@ec2<your-cluster-public-dns>.amazonaws.com -i emr.pem`

**if you get an error related to port 22, you have to add it editing the inbound to the maste node as you made for the port 8998 but instead of using TCP use ssh and chose any user.**

you should see something like that after you connect:

<img src='notebook_ims/emr_screen.png' width=40% />

### Install Python dependencies and download mleap jar
After you have placed the JAR in the right location, please download a couple of necessary dependencies from PyPI. You have to download `boto3` and `mleap`.
You can run the below commands to download the dependencies from PyPI:

`sudo python /usr/lib/python2.7/dist-packages/easy_install.py pip`

You need to have the MLeap JAR in the classpath to be successfully able to use it during model serialization. Please download the JAR (it is an assembly/fat JAR) from the following link using `wget`:

`sudo wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/0.9.6/jar/mleap_spark_assembly.jar /usr/lib/spark/jars`

For the next step only the scikit-learn package is mandatory. If you try to install mleap without installing scikit-learn you will probably get an error which scikit-learn asks for python 3 version. I installed the other packages for pratical reasons.

`sudo /usr/local/bin/pip install paramiko nltk scipy scikit-learn pandas`

So you have to install boto3 and mleap

`sudo pip install boto3`

`sudo pip install mleap`

## Checking that the Spark connection is set up properly
Following the blog mentioned above, we test that the Spark connection setup is done properly by invoking `%%info` in the following cell.

In [1]:
%%info

## Importing PySpark dependencies
Next we will import all the necessary dependencies that will be needed to execute the following cells on our Spark cluster. Please note that we are also importing the `boto3` and `mleap` modules here. 

You need to ensure that the import cell runs without any error to verify that you have installed the dependencies from PyPI properly. Also, this cell will provide you with a valid `SparkSession` named as `spark`.

In [2]:
from __future__ import print_function

import os
import shutil
import boto3

import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from mleap.pyspark.spark_support import SimpleSparkSerializer

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
DATASET_ENCODING = "ISO-8859-1"
TEST_SIZE = 0.3
TRAIN_SIZE = 0.7
VAL_SIZE = 0.2
VOCABULARY_SIZE = 5000
DATA_DIR = 'data'
FEATURES_NUMBER = 50

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#defining schema of the dataset
def get_schema_structure():
    
    #creating schema for dataset
    data_schema = [
        StructField("label", IntegerType(), True),
        StructField("ids", LongType(), True),
        StructField("date", StringType(), True),
        StructField("flag", StringType(), True),
        StructField("user", StringType(), True),
        StructField("text", StringType(), True)
    ]
    return StructType(fields=data_schema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df = spark.read\
.schema(get_schema_structure())\
.format('csv')\
.option('encoding', DATASET_ENCODING)\
.option('header','false')\
.csv('s3a://sagemaker-us-east-2-446439287457/sagemaker/twitter/data/train.csv')\
.select('label', 'text')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:

pre_split_df = df.withColumn("label",
              when(df["label"] == 4, 1).otherwise(df["label"])).select('label', 'text')
train, test = pre_split_df.randomSplit([TRAIN_SIZE, TEST_SIZE], seed=2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Machine Learning task: Predict sentimentfrom review

The dataset is available from [Sentiment140 dataset with 1.6 million tweets](https://www.kaggle.com/kazanova/sentiment140). This is the sentiment140 dataset. It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment . 
The target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive).

We'll use SparkML to pre-process the dataset (apply one or more feature transformers) and train it with the [Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression) algorithm from SparkML.

## Or read data directly from S3


In [8]:
# Please replace the bucket name with your bucket-name and the file-name/key with your file-name/key
df_train = spark.read.parquet('s3a://sagemaker-us-east-2-446439287457/sagemaker/twitter/data/processed/train.parquet')
df_test = spark.read.parquet('s3a://sagemaker-us-east-2-446439287457/sagemaker/twitter/data/processed/test.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_test.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- label: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- data_prep: array (nullable = true)
 |    |-- element: string (containsNull = true)

In [10]:
df_train.where(col('label') == 1).show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------------------+
|label|                text|           data_prep|
+-----+--------------------+--------------------+
|    1|   uploading pict...|[upload, pictur, ...|
|    1|   we break dance...|[we, break, danc,...|
|    1| &quot;I'm a free...|[&quotim, a, free...|
|    1| - Iowa No. 2 in ...|[-, iowa, no, 2, ...|
|    1| Ahhh, what a way...|[ahhh, what, a, w...|
+-----+--------------------+--------------------+
only showing top 5 rows

## Define the feature transformers
The dataset has one categorical column - `target`.

In [13]:
from pyspark.ml.feature import CountVectorizer, NGram, StopWordsRemover, Tokenizer
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="text", outputCol="words")

remover = StopWordsRemover(inputCol="words", outputCol="filtered")
remover.setStopWords(get_stop_words_list())

ngram = NGram(n=2, inputCol="filtered", outputCol="ngrams")
count_vectorizer = CountVectorizer(inputCol="ngrams", outputCol="features").setVocabSize(20000)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Define the Logistic Regression model and perform training
After the data is preprocessed, we define a `LogisticRegression`, define our `Pipeline` comprising of both feature transformation and training stages and train the Pipeline calling `.fit()`.
The parameters have been defined in the notebook LogisticRegression.

In [14]:
lr = LogisticRegression(regParam = 0.001, maxIter = 10, elasticNetParam = 0.3)

pipeline = Pipeline(stages = [ 
    tokenizer,
    remover,
    ngram,
    count_vectorizer,
    lr    
])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
model = pipeline.fit(train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
transformed_train_df = model.transform(train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
transformed_test_df = model.transform(test)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Use the trained `Model` to transform train and validation dataset
Next we will use this trained `Model` to convert our training and validation dataset to see some sample output and also measure the performance scores.The `Model` will apply the feature transformers on the data before passing it to the Random Forest.

In [18]:
transformed_test_df.select('prediction').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|prediction|
+----------+
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
+----------+
only showing top 5 rows

## Evaluating the model on train and validation dataset
Using Spark's `RegressionEvaluator`, we can calculate the `areaUnderROC` on our train and validation dataset to evaluate its performance. 

In [19]:
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
train_eva = evaluator.evaluate(transformed_train_df)
validation_eva = evaluator.evaluate(transformed_test_df)
print("Train areaUnderROC = %g" % train_eva)
print("Validation areaUnderROC = %g" % validation_eva)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Train areaUnderROC = 0.789047
Validation areaUnderROC = 0.774873

## Using `MLeap` to serialize the model
By calling the `serializeToBundle` method from the `MLeap` library, we can store the `Model` in a specific serialization format that can be later used for inference by `sagemaker-sparkml-serving`. 

**If this step fails with an error - `JavaPackage is not callable`, it means you have not setup the MLeap JAR in the classpath properly.**

In [21]:
SimpleSparkSerializer().serializeToBundle(model, "jar:file:/tmp/model.zip", transformed_test_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Convert the model to `tar.gz` format
SageMaker expects any model format to be present in `tar.gz` format, but MLeap produces the model `zip` format. In the next cell, we unzip the model artifacts and store it in `tar.gz` format. 

In [21]:
import zipfile
with zipfile.ZipFile("/tmp/model.zip") as zf:
    zf.extractall("/tmp/model")
    
import tarfile
with tarfile.open("/tmp/model.tar.gz", "w:gz") as tar:
    tar.add("/tmp/model/bundle.json", arcname='bundle.json')
    tar.add("/tmp/model/root", arcname='root')     

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Upload the trained model artifacts to S3
At the end, we need to upload the trained and serialized model artifacts to S3 so that it can be used for inference in SageMaker. 

Please note down the S3 location to where you are uploading your model.

In [22]:
# Please replace the bucket name with your bucket name where you want to upload the model
# Please replace the bucket name with your bucket name where you want to upload the model
s3 = boto3.resource('s3') 
file_name = os.path.join("emr/sentiment/lr/mleap", 'model.tar.gz')
s3.Bucket('sagemaker-us-east-2-446439287457').upload_file('/tmp/model.tar.gz', file_name)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Delete model artifacts from local disk (optional)
If you are training multiple ML models on the same host and using the same location to save the `MLeap` serialized model, then you need to delete the model on the local disk to prevent `MLeap` library failing with an error - `file already exists`.

In [ ]:
import os
os.remove('/tmp/model.zip')
os.remove('/tmp/model.tar.gz')
shutil.rmtree('/tmp/model')

## Hosting the model in SageMaker
Now the second phase of this Notebook begins, where we will host this model in SageMaker and perform predictions against it. 

**For this, please change your kernel to `conda_python2`.** 

### Hosting a model in SageMaker requires two components

* A Docker image residing in ECR.
* a trained Model residing in S3.

For SparkML, Docker image for MLeap based SparkML serving has already been prepared and uploaded to ECR by SageMaker team which anyone can use for hosting. For more information on this, please see [SageMaker SparkML Serving](https://github.com/aws/sagemaker-sparkml-serving-container/). 

MLeap serialized model was uploaded to S3 as part of the Spark job we executed in EMR in the previous steps.

## Creating the endpoint for prediction
Next we'll create the SageMaker endpoint which will be used for performing online prediction. 

For this, we have to create an instance of `SparkMLModel` from `sagemaker-python-sdk` which will take the location of the model artifacts that we uploaded to S3 as part of the EMR job.

### Passing the schema of the payload via environment variable
SparkML server also needs to know the payload of the request that'll be passed to it while calling the `predict` method. In order to alleviate the pain of not having to pass the schema with every request, `sagemaker-sparkml-serving` lets you to pass it via an environment variable while creating the model definitions. 

We'd see later that you can overwrite this schema on a per request basis by passing it as part of the individual request payload as well.

This schema definition should also be passed while creating the instance of `SparkMLModel`.

#### this json format was used to test how to get probabilities

In [1]:
#this schema worked
import json
schema = {
    "input": [
        {
            "name": "text",
            "type": "string"
        }, 
        
    ],
    "output": 
        {
            "name": "probability",
            "type": "float",
            "struct": "vector"
        }
}
schema_json = json.dumps(schema)
print(schema_json)

{"input": [{"type": "string", "name": "text"}], "output": {"type": "float", "name": "probability", "struct": "vector"}}


#### I really wanted to get this two fields but it is not possible. I keep it to the future so we can improve the model a lot adding neutral reviews. Check the Logistic regression results for confusion matrix where probability is specified for more details.

In [78]:
import json
schema = {
    "input": [
        {
            "name": "text",
            "type": "string"
        }, 
        
    ],
    "output": 
        [{
            "name": "prediction",
            "type": "double"
         }, 
          {
            "name": "probability",
            "type": "float",
            "struct": "vector"
          } 
          ]
}
schema_json = json.dumps(schema)
print(schema_json)

{"input": [{"type": "string", "name": "text"}], "output": [{"type": "double", "name": "prediction"}, {"type": "float", "name": "probability", "struct": "vector"}]}


#### so only prediction will be taken

In [2]:
#this schema worked
import json
schema = {
    "input": [
        {
            "name": "text",
            "type": "string"
        }, 
        
    ],
    "output": 
        {
            "name": "prediction",
            "type": "double"
         }
}
schema_json = json.dumps(schema)
print(schema_json)

{"input": [{"type": "string", "name": "text"}], "output": {"type": "double", "name": "prediction"}}


In [3]:
from time import gmtime, strftime
import time

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

import sagemaker
from sagemaker import get_execution_role
from sagemaker.sparkml.model import SparkMLModel

sess = sagemaker.Session()
role = get_execution_role()

# S3 location of where you uploaded your trained and serialized SparkML model
sparkml_data = 's3://{}/{}/{}'.format('sagemaker-us-east-2-446439287457', 'emr/sentiment/lr/mleap', 'model.tar.gz')
model_name = 'sparkml-abalone-' + timestamp_prefix
sparkml_model = SparkMLModel(model_data=sparkml_data, 
                             role=role, 
                             sagemaker_session=sess, 
                             name=model_name,
                             # passing the schema defined above by using an environment 
                             #variable that sagemaker-sparkml-serving understands
                             env={'SAGEMAKER_SPARKML_SCHEMA' : schema_json, 
                                  'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': "application/jsonlines;data=text"})


endpoint_name = 'sparkml-abalone-ep-' + timestamp_prefix
sparkml_model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge', endpoint_name=endpoint_name)

-------------------------------------------------------------------------------------------!

### Invoking the newly created inference endpoint with a payload to transform the data
Now we will invoke the endpoint with a valid payload that `sagemaker-sparkml-serving` can recognize. There are three ways in which input payload can be passed to the request:

* Pass it as a valid CSV string. In this case, the schema passed via the environment variable will be used to determine the schema. For CSV format, every column in the input has to be a basic datatype (e.g. int, double, string) and it can not be a Spark `Array` or `Vector`.

* Pass it as a valid JSON string. In this case as well, the schema passed via the environment variable will be used to infer the schema. With JSON format, every column in the input can be a basic datatype or a Spark `Vector` or `Array` provided that the corresponding entry in the schema mentions the correct value.

* Pass the request in JSON format along with the schema and the data. In this case, the schema passed in the payload will take precedence over the one passed via the environment variable (if any).

#### Passing the payload in CSV format
We will first see how the payload can be passed to the endpoint in CSV format.

In [5]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
payload = "I don't love Tidal :-)"
predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sess, serializer=csv_serializer,
                                content_type=CONTENT_TYPE_CSV, accept='application/jsonlines')
print(predictor.predict(payload))

{"features":[0.5370864647910429,0.46291353520895695]}


#### getting prediction instead

In [37]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
payload = "I don't love it"
predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sess, serializer=csv_serializer,
                                content_type=CONTENT_TYPE_CSV, accept='application/jsonlines')
print(predictor.predict(payload))

0.0


In [9]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
payload = "I love Tidal :-)"
predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sess, serializer=csv_serializer,
                                content_type=CONTENT_TYPE_CSV, accept='application/jsonlines')
print(predictor.predict(payload))

1.0


#### Passing the payload in JSON format
We will now pass a different payload in JSON format.

In [30]:
from sagemaker.content_types import CONTENT_TYPE_NPY
payload = {"data": ["Tidal is amazing kkkk"]}
predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sess, serializer=json_serializer,
                                content_type=CONTENT_TYPE_JSON, accept=CONTENT_TYPE_CSV)

print(predictor.predict(payload))

0.4491398963389081,0.550860103661092


#### testing prediction

In [5]:
from sagemaker.content_types import CONTENT_TYPE_NPY
payload = {"data": ["Tidal is cool kkkk"]}
predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sess, serializer=json_serializer,
                                content_type=CONTENT_TYPE_JSON, accept=CONTENT_TYPE_CSV)

print(predictor.predict(payload))

1.0


#### Passing the payload with both schema and the data
Next we will pass the input payload comprising of both the schema and the data. 
Here i will create two payloads, with one I get the probability and with the another one I get the prediction.

In [42]:
payload_prob = {
    "schema": {
        "input": [
        {
            "name": "text",
            "type": "string"
        }, 
    ],
    "output": 
        {
            "name": "probability",
            "type": "float",
            "struct": "vector"
          }
    },
    "data": ["tidal is not sad"]
}

predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sess, serializer=json_serializer,
                                content_type=CONTENT_TYPE_JSON, accept=CONTENT_TYPE_CSV)

print(predictor.predict(payload_prob))

0.4390124579424653,0.5609875420575346


In [30]:
payload_pred = {
    "schema": {
        "input": [
        {
            "name": "text",
            "type": "string"
        }, 
    ],
    "output": 
        {
            "name": "prediction",
            "type": "double"
         }
    },
    "data": ["I don't love it "]
}

predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sess, serializer=json_serializer,
                                content_type=CONTENT_TYPE_JSON, accept=CONTENT_TYPE_CSV)

print(predictor.predict(payload_pred))

0.0


In [41]:
payload_prob = {
    "schema": {
        "input": [
        {
            "name": "text",
            "type": "string"
        }, 
    ],
    "output": 
        {
            "name": "probability",
            "type": "float",
            "struct": "vector"
          }
    },
    "data": ["I am mad"]
}

predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sess, serializer=json_serializer,
                                content_type=CONTENT_TYPE_JSON, accept=CONTENT_TYPE_CSV)

print(predictor.predict(payload_prob))

0.4491398963389081,0.550860103661092


### Deleting the Endpoint (Optional)
Next we will delete the endpoint so that you do not incur the cost of keeping it running.

In [ ]:
boto_session = sess.boto_session
sm_client = boto_session.client('sagemaker')
sm_client.delete_endpoint(EndpointName=endpoint_name)

I am not currently using this transformer because I had problems with wordnet

In [12]:
 def get_stop_words_list():
            stop_words_list = ['link','google','facebook','yahoo','rt','i', 'me', 'my', 'myself', 'tag'
                              'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll",
                              "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his',
                              'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its',
                              'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which',
                              'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
                              'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do',
                              'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because',
                              'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against',
                              'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below',
                              'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again',
                              'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all',
                              'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such',
                              'only', 'own', 'same', 'so', 'than', 'too', 's', 't', 'can', 'will',
                              'just', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've',
                              'y', 'ain', 'ma', 'u', 'aren', 'ø', 'å', 'æ', 'b', 'c', 'd', 'e']

            return stop_words_list

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Important notes
Since the notebook is not running locally you can not import the python classes as we did in the Data preparation notebook.

In [ ]:
#here you can check the files presents in your dir
import os

path = os.getcwd()

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

you can acces the files via local mode

In [ ]:
%%local

import os

path = os.getcwd()

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

the question is how to send this modules to spark?